In [ ]:
import json
from fix_label import fix_general_label_error
import random

In [ ]:
data_set_folder = "data/multi-woz/"
file_train = data_set_folder + 'train_dials.json'
file_dev = data_set_folder + 'dev_dials.json'
file_test = data_set_folder+ 'test_dials.json'
try:
    fp_ont = data_set_folder+"MULTIWOZ2 2/ontology.json"
    data_ont = json.load(open(fp_ont, 'r'))
    data_ont_full = json.load(open(fp_ont, 'r'))
except:
    fp_ont = data_set_folder+"ontology.json"
    data_ont = json.load(open(fp_ont, 'r'))
    data_ont_full = json.load(open(fp_ont, 'r'))
    
### Read ontology file
ontology = {}
for domain_slot in data_ont:
    domain, slot = domain_slot.split('-')
    if domain not in ontology:
        ontology[domain] = {}
    ontology[domain][slot] = {}
    for value in data_ont[domain_slot]:
        ontology[domain][slot][value] = 1
        
### Read woz logs and write to tsv files
fp_train = open(data_set_folder+"train_gluelike.tsv", "w")
fp_dev = open(data_set_folder+"dev_gluelike.tsv", "w")
fp_test = open(data_set_folder+"test_gluelike.tsv", "w")

fp_train.write('# Dialogue ID\tTurn Index\tUser Utterance\tSystem Response\t')
fp_dev.write('# Dialogue ID\tTurn Index\tUser Utterance\tSystem Response\t')
fp_test.write('# Dialogue ID\tTurn Index\tUser Utterance\tSystem Response\t')

SLOTS = []
for domain in sorted(ontology.keys()):
    for slot in sorted(ontology[domain].keys()):
        temp_slot = slot.replace(" ","").lower() if ("book" not in slot) else slot.lower()
        SLOTS.append(domain+"-"+temp_slot)
        fp_train.write(str(domain) + '-' + str(slot) + '\t')
        fp_dev.write(str(domain) + '-' + str(slot) + '\t')
        fp_test.write(str(domain) + '-' + str(slot) + '\t')    
    
fp_train.write('\n')
fp_dev.write('\n')
fp_test.write('\n')

for file_name in ["train", "dev", "test"]:
    fp_out = globals()["fp_{}".format(file_name)]
    
    dials = json.load(open(globals()["file_{}".format(file_name)], "r"))
    
    for dial_idx, dial_dict in enumerate(dials): 
        for idx, turn in enumerate(dial_dict["dialogue"]):
            assert idx == int(turn["turn_idx"])
            user_utterance = turn["transcript"].strip()
            user_utterance = user_utterance.replace('\t', ' ')
            user_utterance = user_utterance.replace('\n', ' ')
            user_utterance = user_utterance.replace('  ', ' ')
            
            system_response = turn["system_transcript"].strip()
            system_response = system_response.replace('\t', ' ')
            system_response = system_response.replace('\n', ' ')
            system_response = system_response.replace('  ', ' ')

            fp_out.write(str(dial_dict["dialogue_idx"]))                   # 0: dialogue ID
            fp_out.write('\t' + str(turn["turn_idx"]))           # 1: turn index
            fp_out.write('\t' + str(user_utterance))     # 2: user utterance
            fp_out.write('\t' + str(system_response))    # 3: system response
            
            belief = fix_general_label_error(turn["belief_state"], False, SLOTS)
            
            for key, value in belief.items():
                domain, slot = key.split("-")
                
                if value == "doesn't care" or value == "don't care" or value == "dontcare" or \
                    value == "dont care" or value == "does not care" or value == "do not care": 
                    belief[key] = "do n't care"
                
            for domain in sorted(ontology.keys()):
                for slot in sorted(ontology[domain].keys()):
                    
                    temp_slot = slot.replace(" ","").lower() if ("book" not in slot) else slot.lower()
                    domain_slot = domain + "-" + temp_slot
                    
                    if domain_slot in belief:
                        fp_out.write('\t' + belief[domain_slot])

                        if belief[domain_slot] not in data_ont_full[domain+"-"+slot]:
                            print("value (%s) in domain (%s) slot (%s) is not defined in ontology" 
                                  % (belief[domain_slot], domain, slot))
                            data_ont_full[domain+"-"+slot].append(belief[domain_slot])
                    else:
                        fp_out.write('\tnone')

            fp_out.write('\n')
    
    fp_out.flush()

fp_train.close()
fp_dev.close()
fp_test.close()

json.dump(data_ont_full, open(fp_ont.replace(".json", "_full.json"), "w"), indent=4)
